# 当代人工智能实验一：文本分类
## ——TF-IDF & Decision Tree

### 一. 引入必要模块
numpy将用于数据的处理。
time用于记录代码运行时间。
DecisionTreeClassifier用于运行决策树。
train_test_split用于进行训练集与验证集的划分。
classification_report用于衡量模型的训练表现。

In [1]:
import numpy as np
import time
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

### 二. 载入数据

In [2]:
labelList = np.load('labelList.npy')
textList = np.load('textVectorList.npy')
result_list = np.load('result_list.npy')

### 三. 利用训练集数据训练决策树模型
使用sklearn中的DecisionTreeClassifier模块。
在这里，我们通过蒙特卡洛交叉验证来验证模型的正确性。

In [3]:
# 划分训练集与测试集，这里选取12.5%的数据作为测试集，剩余数据作为训练集。
# random_state数值是不同会让训练集与测试集不同，若写为None则每次都随机生成。
accuracyTotal = 0
precisionTotal = 0
recallTotal = 0
f1Total = 0
LOOP_NUMBER = 5
target_names = ['class_0', 'class_1', 'class_2', 'class_3', 'class_4', 'class_5', 'class_6', 'class_7', 'class_8', 'class_9']

start_time = time.time()
for loop in range(LOOP_NUMBER):
    text_train, text_test, label_train, label_test = train_test_split(textList, labelList, test_size=0.125, random_state=None)
    dt_classifier = DecisionTreeClassifier(random_state=42)
    dt_classifier.fit(text_train, label_train)
    y_pred = dt_classifier.predict(text_test)
    accuracy = dt_classifier.score(text_test, label_test)
    classification_rep = classification_report(label_test, y_pred, target_names=target_names, output_dict=True)
    # 提取相应的指标值
    precision = classification_rep['weighted avg']['precision']
    recall = classification_rep['weighted avg']['recall']
    f1 = classification_rep['weighted avg']['f1-score']

    accuracyTotal += accuracy
    precisionTotal += precision
    recallTotal += recall
    f1Total += f1

end_time = time.time()
# 计算平均值
runtime = (end_time - start_time) / LOOP_NUMBER
accuracy_avg = accuracyTotal / LOOP_NUMBER
precision_avg = precisionTotal / LOOP_NUMBER
recall_avg = recallTotal / LOOP_NUMBER
f1_avg = f1Total / LOOP_NUMBER

print("平均运行时间为：", runtime)
print("模型准确率：", accuracy_avg)
print("模型精确度：", precision_avg)
print("模型召回率：", recall_avg)
print("模型F1-score：", f1_avg)

平均运行时间为： 39.13006644248962
模型准确率： 0.7867999999999999
模型精确度： 0.7888546919511834
模型召回率： 0.7867999999999999
模型F1-score： 0.7866261113542182


我们发现，其分类表现较差。这可能是由于我们没有进行任何参数调整所导致的，因此我们将调整个别参数来观察运行效果。
然而，通过对决策树可以调整的参数进行研究，我们发现，调节max_depth, min_samples_split, min_samples_leaf, max_leaf_nodes等参数，通常只是为了抑制模型的复杂性与过拟合。我们的模型目前遇到的最大问题为，预测的效果非常不好，调整上述参数对于优化模型的预测效果不会有显著效果，因此我们主要调整了衡量分裂质量的标准，将criterion改为了entropy.

In [8]:
# 划分训练集与测试集，这里选取12.5%的数据作为测试集，剩余数据作为训练集。
# random_state数值是不同会让训练集与测试集不同，若写为None则每次都随机生成。
accuracyTotal = 0
precisionTotal = 0
recallTotal = 0
f1Total = 0
LOOP_NUMBER = 5
target_names = ['class_0', 'class_1', 'class_2', 'class_3', 'class_4', 'class_5', 'class_6', 'class_7', 'class_8', 'class_9']

start_time = time.time()
for loop in range(LOOP_NUMBER):
    text_train, text_test, label_train, label_test = train_test_split(textList, labelList, test_size=0.125, random_state=None)
    dt_classifier = DecisionTreeClassifier(criterion='entropy', random_state=42)
    dt_classifier.fit(text_train, label_train)
    y_pred = dt_classifier.predict(text_test)
    accuracy = dt_classifier.score(text_test, label_test)
    classification_rep = classification_report(label_test, y_pred, target_names=target_names, output_dict=True)
    # 提取相应的指标值
    precision = classification_rep['weighted avg']['precision']
    recall = classification_rep['weighted avg']['recall']
    f1 = classification_rep['weighted avg']['f1-score']

    accuracyTotal += accuracy
    precisionTotal += precision
    recallTotal += recall
    f1Total += f1

end_time = time.time()
# 计算平均值
runtime = (end_time - start_time) / LOOP_NUMBER
accuracy_avg = accuracyTotal / LOOP_NUMBER
precision_avg = precisionTotal / LOOP_NUMBER
recall_avg = recallTotal / LOOP_NUMBER
f1_avg = f1Total / LOOP_NUMBER

print("平均运行时间为：", runtime)
print("模型准确率：", accuracy_avg)
print("模型精确度：", precision_avg)
print("模型召回率：", recall_avg)
print("模型F1-score：", f1_avg)

平均运行时间为： 31.50426330566406
模型准确率： 0.7426
模型精确度： 0.7496185196236086
模型召回率： 0.7426
模型F1-score： 0.7441254927342695


发现最后模型的预测效果仍然不尽理想。
实际上，决策树模型被认为不是最合适文本分类任务的一个模型。因此，我们不选择使用决策树模型作为最终使用的分类模型。